In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import sqlite3

# Conectar ao banco de dados e carregar os dados
db_path = '/Users/leonardooliveira/Desktop/PBI-EXAI/Projeto-BI/preparacao/Experimentai.db'
conn = sqlite3.connect(db_path)
assinantes_df = pd.read_sql_query("SELECT * FROM fato_assinantes", conn)
dim_assinantes = pd.read_sql_query("SELECT * FROM dim_demografico_assinantes", conn)
pagamentos_df = pd.read_sql_query("SELECT * FROM dim_pagamentos", conn)
coupons_df = pd.read_sql_query("SELECT * FROM dim_coupons", conn)

# Mesclar tabelas e tratar valores nulos
assinantes = (
    assinantes_df
    .merge(pagamentos_df, left_on='id_pagamento', right_on='id_pagamento', how='left')
    .merge(coupons_df, left_on='coupon', right_on='id_cupom', how='left')
    .merge(dim_assinantes, on='id_demografico_assinantes', how='left')
)

# Converter colunas de data para datetime
assinantes['created_utc'] = pd.to_datetime(assinantes['created_utc'], errors='coerce')
assinantes = assinantes.dropna(subset=['created_utc'])  # Remover linhas com datas inválidas

# Criar variável de tendência numérica baseada no tempo
assinantes['date_numeric'] = (assinantes['created_utc'] - assinantes['created_utc'].min()).dt.days

# Agrupar dados para obter o número de assinantes por dia
assinantes_daily = (
    assinantes.groupby('date_numeric')
    .size()
    .reset_index(name='active_subscribers')
)

# Adicionar variáveis de tendência
assinantes_daily['trend'] = assinantes_daily['date_numeric']
assinantes_daily['trend_squared'] = assinantes_daily['trend'] ** 2

# Adicionar variáveis de sazonalidade
assinantes_daily['month'] = assinantes['created_utc'].dt.month
assinantes_daily['day_of_week'] = assinantes['created_utc'].dt.dayofweek
assinantes_daily['is_weekend'] = assinantes_daily['day_of_week'].isin([5, 6]).astype(int)

# Dividir os dados entre features (X) e variável dependente (y)
X = assinantes_daily[['trend', 'trend_squared', 'month', 'day_of_week', 'is_weekend']]
y = assinantes_daily['active_subscribers']

# Dividir dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo de regressão linear múltipla
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'R²: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

# Visualizar os resultados
plt.figure(figsize=(12, 6))
plt.scatter(X_test['trend'], y_test, label='Real', color='blue')
plt.scatter(X_test['trend'], y_pred, label='Predito', color='red')
plt.xlabel('Dias')
plt.ylabel('Número de Assinantes Ativos')
plt.title('Regressão Linear Múltipla: Previsão de Assinantes')
plt.legend()
plt.grid(True)
plt.show()

# Fechar conexão com o banco de dados
conn.close()


KeyError: 'id_pagamento'